Read the .res file

In [79]:
%load_ext autoreload
%autoreload 2
import json
from scipy import sparse
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import string
from sys import getsizeof
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Create id to docid dict; write to file

In [4]:
# %store -r dataset
# dataset.shape[0]

dataset = pd.read_json("../data/rcv1_news.json", lines=True)
dataset.shape[0]

804414

In [5]:
getsizeof(dataset)

1506249058

In [10]:
 dataset

,categories,content,date,headline,id,lang,title
0,"[E11, ECAT, M11, M12, MCAT]",Emerging evidence that Mexico's economy was ba...,1996-08-20,Recovery excitement brings Mexican markets to ...,2286,en,MEXICO: Recovery excitement brings Mexican mar...
1,"[C24, CCAT]",Chrysler Corp. Tuesday announced $380 million ...,1996-08-20,Chrysler plans new investments in Latin America.,2287,en,USA: Chrysler plans new investments in Latin A...
2,"[C15, C151, CCAT, E41, ECAT, GCAT, GJOB]",CompuServe Corp. Tuesday reported a surprising...,1996-08-20,"CompuServe reports loss, cutting work force.",2288,en,"USA: CompuServe reports loss, cutting work force."
3,"[C15, C151, CCAT]",CompuServe Corp. Tuesday reported a surprising...,1996-08-20,"CompuServe reports loss, cutting work force.",2289,en,"USA: CompuServe reports loss, cutting work force."
4,"[C11, C22, CCAT]",If dining at Planet Hollywood made you feel li...,1996-08-20,Planet Hollywood launches credit card.,2290,en,USA: Planet Hollywood launches credit card.
5,"[M14, MCAT]",Hog prices fell Tuesday after government slaug...,1996-08-20,"Hog prices tumble as supplies increase, cocoa ...",2291,en,"USA: Hog prices tumble as supplies increase, c..."
6,"[M11, M12, M13, M132, M14, MCAT]",Blue-chip stocks rallied Tuesday after the Fed...,1996-08-20,Blue chips end up as Fed keeps interest rates ...,2292,en,USA: Blue chips end up as Fed keeps interest r...
7,"[C22, CCAT]",Sprint Corp. Tuesday announced plans to offer ...,1996-08-20,Sprint to offer consumer Internet access service.,2293,en,USA: Sprint to offer consumer Internet access ...
8,"[E14, ECAT]",Shoppers are loading up this year on perennial...,1996-08-20,Back-to-school spending is up.,2294,en,USA: Back-to-school spending is up.
9,"[C12, CCAT, GCAT, GCRIM]",Kansas and Arizona filed lawsuits against some...,1996-08-20,"Kansas, Arizona add to suits against tobacco f...",2295,en,"USA: Kansas, Arizona add to suits against toba..."


In [15]:
print(len(list(dataset["id"])))
sorted(list(dataset["id"]))
print(len(set(dataset["id"])))

804414
804414


In [25]:
queryTitleDoc = defaultdict()
for i in range(dataset.shape[0]):
    docid = dataset["id"].iloc[i]
    queryTitleDoc[int(i)] = int(docid)

In [30]:
print(len(queryTitleDoc))
queryTitleDoc[0]

804414


2286

In [31]:
with open('../data/queryTitleDoc-RCV1.json', 'w') as fp:
    json.dump(queryTitleDoc, fp)

# 2. READ query types from the 2 search engines

## Query types on Elastic

* ('A', 238580)
* ('C', 152506)
* ('B', 228059)
* ('D', 103161)
* No docs in top 1000: 82,108

Out of a total of 804414 we have the 4 types of queries summing up to 722,306.

In [32]:
with open('../data/RCV1-queryTypeElastic.json') as json_data:
    elastic_query_types = json.load(json_data)

In [39]:
print(elastic_query_types.keys())
print(len(elastic_query_types["A"]))

dict_keys(['A', 'C', 'B', 'D'])
238580


## Query types on Terrier

* ('A', 225227)
* ('C', 152174)
* ('B', 221300)
* ('D', 105383)
* No docs in top 1000 results: 100,330

Out of a total of 804414 we have the 4 types of queries summing up to 704,084. 


In [35]:
with open('../data/RCV1-queryTypeTerrier.json') as json_data:
    terrier_query_types = json.load(json_data)

In [38]:
print(terrier_query_types.keys())
print(len(terrier_query_types["A"]))

dict_keys(['A', 'C', 'B', 'D'])
225227


# 3. Get global queries - all

In [43]:
# Checks
print(len(set(elastic_query_types["A"]).intersection(set(elastic_query_types["D"]))))

print(type(elastic_query_types))
for key, itemlist in elastic_query_types.items():
    elastic_query_types[key] = sorted(list(itemlist))
    print(key, len(itemlist))
# the numbers are ok

print(type(elastic_query_types["A"]))
print(elastic_query_types["A"][:10])
print(terrier_query_types["A"][:10])

0
<class 'dict'>
A 238580
C 152506
B 228059
D 103161


## We intersect the 2 SE

- we intersect A,B,C,D and decide upon the final queries
- we run there queries on BM25P10 - idf and tfidf and count:
    - how many intersect in A
    - how many moved from B to A
    - how many move from C to A and B and so on


In [46]:
# let's do some intersections
global_query_types = defaultdict()
for keyT, itemlistT in terrier_query_types.items():
    itemlistE = elastic_query_types[keyT]
    itemlistG = sorted(list(set(itemlistT) & set(itemlistE)))
    print(keyT)
    print("Terrier: ", len(itemlistT), " Elastic: ", len(itemlistE), " Global: ",len(itemlistG))
    global_query_types[keyT] = itemlistG

A
Terrier:  225227  Elastic:  238580  Global:  159140
C
Terrier:  152174  Elastic:  152506  Global:  67640
B
Terrier:  221300  Elastic:  228059  Global:  120874
D
Terrier:  105383  Elastic:  103161  Global:  45048


In [48]:
# save global
with open('../data/RCV1-queryTypeGlobal.json', 'w') as fp:
    json.dump(global_query_types, fp)

# 4. Double check the Global and sample 10.000 queries per type

In [50]:
with open('../data/RCV1-queryTypeGlobal.json') as json_data:
    global_q = json.load(json_data)

In [65]:
for k,v in global_q.items():
    print(k, len(v))

A 159140
C 67640
B 120874
D 45048


In [74]:
for qType in ["A","B","C","D"] :  
    sample_10000 = random.sample(global_q[qType], 15000)
    with open("../data/RCV1-sample-15000-"+ qType + ".txt", 'w') as fp:
        json.dump(sample_10000, fp) # we are writing a list

In [75]:
with open('../data/RCV1-sample-15000-A.txt') as json_data:
    sample10000A = json.load(json_data)
    print(len(sample10000A))
    print(sample10000A[:20])
#     print(min(sample10000A))
#     print(min(global_q["A"]))

15000
[553395, 220299, 153541, 331770, 515413, 85178, 152323, 275538, 487672, 113786, 145976, 642508, 9497, 414209, 12990, 625431, 280859, 296564, 162988, 48197]


# 5. Read query type ids and generate Title query files

In [58]:
newTitle = dataset.headline.str.replace('[^a-zA-Z ]', '')

In [59]:
newTitle[0]

'Recovery excitement brings Mexican markets to life'

In [76]:
def gen_sample_queries(newTitle, qtype):
    sampleFilename = "../data/RCV1-sample-15000-" + qtype + ".txt"
    with open(sampleFilename) as json_data:
        sample_10000 = json.load(json_data)
    with open("../data/RCV1-title-queries-sample-15000-"+ qtype + ".txt", "w") as f:
        for qid in sample_10000:
            f.write(str(qid)+":"+newTitle[qid]+"\n")

In [77]:
for qType in ["A","B","C","D"] :
    gen_sample_queries(newTitle, qType)

# 6. Generate query groups according to HARDNESS

In [80]:
store -r query_hardness_dict

In [119]:
# Split method 1 - 3 equal parts

query_hardness_list = list(query_hardness_dict.items())
sorted_query_hardness_list = sorted(query_hardness_list, key=lambda x: x[1])

a,b,c = np.array_split(np.array(sorted_query_hardness_list),3)

print(len(a),len(b), len(c))
# np.set_printoptions(suppress=True)
print(a[-1], b[-1], c[-1])
print(a[:,0])

268138 268138 268138
[ 753837.               0.03428571] [ 422106.               0.06976744] [ 793247.       7.]
[ 278632.  379065.  569182. ...,  751894.  752381.  753837.]


In [123]:
# We look at the intervals from above
h = a[:,0].astype(int).tolist()
m = b[:,0].astype(int).tolist()
e = c[:,0].astype(int).tolist()

print(len(h),len(m), len(e))

268138 268138 268138


In [131]:
for qType in ["A","B","C","D"] :
    print(qType)
    group_intersect_H = set(global_q[qType]) & set(h)
    group_intersect_M = set(global_q[qType]) & set(m)
    group_intersect_E = set(global_q[qType]) & set(e)
    
    print(len(group_intersect_H))
    print(len(group_intersect_M))
    print(len(group_intersect_E))
    
    sample_H1000 = random.sample(group_intersect_H, 1000)
    sample_M1000 = random.sample(group_intersect_M, 1000)
    sample_E1000 = random.sample(group_intersect_E, 1000)
    
    print(set(sample_H1000) & set(sample_M1000))

#     with open("../data/RCV1-sample-H1000-"+ qType + ".txt", 'w') as fp:
#         json.dump(sample_H1000, fp) # we are writing a list
#     with open("../data/RCV1-sample-M1000-"+ qType + ".txt", 'w') as fp:
#         json.dump(sample_M1000, fp) # we are writing a list
#     with open("../data/RCV1-sample-E1000-"+ qType + ".txt", 'w') as fp:
#         json.dump(sample_E1000, fp) # we are writing a list

A
35303
62091
61746
set()
B
40398
39612
40864
set()
C
25846
21995
19799
set()
D
17958
14963
12127
set()


In [128]:
def gen_sample_queries(newTitle, qtype, hardness):
    sampleFilename = "../data/RCV1-sample-"+hardness+"1000-" + qtype + ".txt"
    with open(sampleFilename) as json_data:
        sample_1000 = json.load(json_data)
    with open("../data/RCV1-title-queries-sample-"+hardness+"1000-"+ qtype + ".txt", "w") as f:
        for qid in sample_1000:
            f.write(str(qid)+":"+newTitle[qid]+"\n")

In [129]:
for qType in ["A","B","C","D"] :
    for hardness in ["E", "M", "H"]:
        gen_sample_queries(newTitle, qType, hardness)